In [157]:
import pandas as pd
import numpy as np
from time import time
import json
from sqlalchemy import create_engine
import re
import datetime
from mpl_toolkits.basemap import Basemap

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.style.use('seaborn-whitegrid')
import numpy as np
from mpl_toolkits.basemap import Basemap
from geopy.distance import distance as dist
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import folium

In [158]:
# Set up the SQL connection
dbname = 'redshift'
url = 'panamera-ods.cnsuxis6zqxr.us-west-2.redshift.amazonaws.com:5439/panameraods'
engine = create_engine("postgresql://" + 'lucasdresl' + ":" + 'cQsY801a76x82x62aWDT' + "@" + url)


In [72]:
query = """

drop table if exists active_listings;
select
  id
 into temp table active_listings
from livesync.panamera_ads
where status='active'
  and livesync_dbname='olxin'
group by 1


;drop table if exists location_of_item;
select
   ad_id,
   map_lat,
   map_lon,
   state,
   city,
   district
    into temp table location_of_item
  from livesync.panamera_locations a
   join active_listings b on a.ad_id=b.id
where livesync_dbname='olxin'

;drop table if exists distribution_active_listings;
select
 map_lat,
 map_lon,
 count(distinct ad_id) as qty_listings
  into temp table distribution_active_listings
   from location_of_item
group by 1,2
order by 3 desc

;drop table if exists distribution_active_listings_with_location_tree;
select
 a.*,
 geography_level , geography_l1_name_en,geography_l2_name_en,geography_l3_name_en
   into temp table distribution_active_listings_with_location_tree
  from distribution_active_listings a
   left join (select geography_latitude , geography_longitude ,geography_level , geography_l1_name_en,geography_l2_name_en,geography_l3_name_en from ods.dim_geographies where country_sk='olx|asia|in' ) b on a.map_lat=b.geography_latitude and a.map_lon=b.geography_longitude
order by qty_listings desc;


select * from distribution_active_listings_with_location_tree 

"""

In [30]:
query = """

drop table if exists active_listings;
select
  id
 into temp table active_listings
from livesync.panamera_ads
where status='active'
  and livesync_dbname='olxin'
group by 1


;drop table if exists location_of_item;
select
   ad_id,
   map_lat,
   map_lon,
   state,
   city,
   district
    into temp table location_of_item
  from livesync.panamera_locations a
   join active_listings b on a.ad_id=b.id
where livesync_dbname='olxin'


;drop table if exists distribution_active_listings_with_location_tree;
select
 a.*,
 geography_level , geography_l1_name_en,geography_l2_name_en,geography_l3_name_en
   into temp table distribution_active_listings_with_location_tree
  from location_of_item a
   left join (select geography_latitude , geography_longitude ,geography_level , geography_l1_name_en,geography_l2_name_en,geography_l3_name_en from ods.dim_geographies where country_sk='olx|asia|in' ) b on a.map_lat=b.geography_latitude and a.map_lon=b.geography_longitude
 ;


select
* 
from distribution_active_listings_with_location_tree
order by random() limit 500000

"""

In [76]:
df = pd.read_sql_query(query.replace('%', '%%'), engine)


In [77]:
df.head()

,map_lat,map_lon,qty_listings,geography_level,geography_l1_name_en,geography_l2_name_en,geography_l3_name_en
0,29.865970,77.878790,143,3.0,Uttaranchal,Roorkee,BSM Tiraha
1,13.026661,77.590791,80,3.0,Karnataka,Bengaluru,Ganga Nagar
2,30.872740,75.872032,55,NaN,None,None,None
3,22.460190,70.044450,42,3.0,Gujarat,Jamnagar,Satyam Colony
4,23.569180,87.237070,33,3.0,West Bengal,Durgapur,Gopalmath


In [78]:
df = df.sort_values(['qty_listings'] , ascending = False)


In [79]:
df['total_listings'] = df['qty_listings'].sum()
df['porcentaje'] = df['qty_listings'] / df['total_listings']


In [84]:
df.head()

,map_lat,map_lon,qty_listings,geography_level,geography_l1_name_en,geography_l2_name_en,geography_l3_name_en,total_listings,porcentaje
283117,19.134900,72.825360,4777,3.0,Maharashtra,Mumbai,Andheri West,3419387,0.001397
12484,28.619064,77.053974,4119,3.0,Delhi,Delhi,Uttam Nagar,3419387,0.001205
78094,19.115492,72.872696,4087,3.0,Maharashtra,Mumbai,Andheri East,3419387,0.001195
298849,0.000000,0.000000,3819,NaN,None,None,None,3419387,0.001117
76295,28.619064,77.053978,2750,NaN,None,None,None,3419387,0.000804


In [177]:
df.replace({None: 'Not in location tree'} , inplace=True)
df_1000 = df[:500]
colordict = {1: 'blue', 2: 'lightblue', 3: '#930000' , 'Not in location tree':'#ff9090'}

# Make an empty map
m = folium.Map(location=[22.576588, 79.397677], zoom_start=4)

for lat, lon,traffic_q ,vol, l1, l3 in zip(df_1000['map_lat'], df_1000['map_lon'],df_1000['geography_level'], df_1000['qty_listings'], df_1000['geography_l1_name_en'], df_1000['geography_l3_name_en']):
    folium.CircleMarker(
        [lat, lon],
        radius=vol/200,
        popup = ('L1: ' + str(l1).capitalize() + '<br>'
                 'L3: ' + str(l3) + '<br>'
                 'Listings: ' + str(vol)
                ),
        color='crimson',
        key_on = traffic_q,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[traffic_q],
        fill=True,
        fill_opacity=0.7
        ).add_to(m)
m

In [164]:
query = """

drop table if exists lat_and_long_searches;
select
 left(params_lat , 9) as params_lat,
 left(params_long , 9) as params_long ,
 params_resultset_id
  into temp table lat_and_long_searches
from spectrum.hydra_p_olx_android
where
    params_en = 'listings_results'
and year = 2019 and month=10 and day=10 and params_cc = 'in'
group by 1,2,3
order by 3 desc

-- Identificacion de las busquedas
;drop table if exists busquedas2;
SELECT
   replace(replace(json_extract_path_text(filters, 'location', 'id', true) , '[' , '') , ']' , '')   AS location_type,
   h.session_long,
   h.date_event_nk,
   resultset_id,
   origin_nk
 into temp table busquedas2
FROM ods.panameraolx_asia_hydra_ninja_android_last_month h
WHERE country_sk = 'olx|asia|in'
AND trackevent = 'listings_results'
and date_event_nk = '2019-10-10'
group by 1,2,3,4,5

-- Group of all other tables
;drop table if exists lastest_table;
select
    case
        when a.location_type = '1000001' then 'India'
        when a.location_type != '1000001' and len(a.location_type) > 4 then 'Polygon'
        when a.location_type = '' then 'Current_location' end as locations,
    a.location_type,
    a.date_event_nk,
    a.session_long,
    a.resultset_id,
    a.origin_nk
  into temp table lastest_table
    from busquedas2 a
    ;

drop table if exists last;
select 
a.*,
c.locations,
c.location_type
 into temp table last
from lat_and_long_searches a
 left join (select locations,location_type ,resultset_id from lastest_table group by 1,2,3) c on a.params_resultset_id=c.resultset_id

;

select 
 *
from last a
 left join (select geography_latitude , geography_longitude ,geography_level ,geography_nk , geography_l1_name_en,geography_l2_name_en,geography_l3_name_en from ods.dim_geographies where country_sk='olx|asia|in')  b on a.location_type=b.geography_nk
order by random()
limit 500000

"""

In [165]:
dfs = pd.read_sql_query(query.replace('%', '%%'), engine)

In [166]:
#79.567554
def lat(df):
    aux_list = []
    for x in range(0,len(df.locations)):
        if df['locations'].iloc[x] == 'Polygon':
            aux_item = df['geography_latitude'].iloc[x]
        elif df['locations'].iloc[x] == 'Current_location':
            aux_item = df['params_lat'].iloc[x]
        elif df['locations'].iloc[x] == 'India':
            aux_item = 22.444716
        else:
            aux_item = None
        aux_list.append(aux_item)
    return aux_list
        
    
def long(df):
    aux_list = []
    for x in range(0,len(df.locations)):
        if df['locations'].iloc[x] == 'Polygon':
            aux_item = df['geography_longitude'].iloc[x]
        elif df['locations'].iloc[x] == 'Current_location':
            aux_item = df['params_long'].iloc[x]
        elif df['locations'].iloc[x] == 'India':
            aux_item = 22.444716
        else:
            aux_item = None
        aux_list.append(aux_item)
    return aux_list


In [167]:
dfs['lat'] = pd.Series(lat(dfs))
dfs['long'] = pd.Series(long(dfs))

In [278]:
dfs[dfs['lat'] == '28.67']

,params_lat,params_long,params_resultset_id,locations,location_type,geography_latitude,geography_longitude,geography_level,geography_nk,geography_l1_name_en,geography_l2_name_en,geography_l3_name_en,lat,long,uno


In [168]:
dfs['geography_l1_name_en'] = dfs['geography_l1_name_en'].replace({None:'Current_location'})
dfs['geography_l3_name_en'] = dfs['geography_l3_name_en'].replace({None:'Current_location'})
dfs['uno'] = 1
dfs.head(20)

,params_lat,params_long,params_resultset_id,locations,location_type,geography_latitude,geography_longitude,geography_level,geography_nk,geography_l1_name_en,geography_l2_name_en,geography_l3_name_en,lat,long,uno
0,None,None,16db5232ee3xe6bc2948|1570702224091,India,1000001,NaN,NaN,NaN,None,Current_location,None,Current_location,22.4447,22.4447,1
1,19.169371,72.870563,16d32bf8b93x7d404be5|1570741674415,None,None,NaN,NaN,NaN,None,Current_location,None,Current_location,None,None,1
2,13.124734,80.215225,16b97a42c8fx15a285e2|1570698181833,Polygon,4400983,11.84842,77.75032,2.0,4400983,Tamil Nadu,Kolathur,Unknown,11.8484,77.7503,1
3,22.309429,72.136230,16b30c84453xa5e600ca|1570708631184,Polygon,2001154,22.30943,72.13623,1.0,2001154,Gujarat,Unknown,Unknown,22.3094,72.1362,1
4,26.905658,75.749893,16db7cd82c4x34c95b91|1570747323649,None,None,NaN,NaN,NaN,None,Current_location,None,Current_location,None,None,1
5,26.728777,94.212033,16bc4d43bfcx1d5d83ad|1570701946959,Polygon,5325529,18.55233,73.82761,3.0,5325529,Maharashtra,Pune,Pune Vidyapith,18.5523,73.8276,1
6,16.303764,80.438041,16b41f0c91fx247c7ddd|1570702039790,Polygon,4058524,16.31531,80.44532,2.0,4058524,Andhra Pradesh,Guntur,Unknown,16.3153,80.4453,1
7,None,None,16d5eaf29bax5b8d6ed7|1570717749144,Polygon,4059014,18.53121,73.85799,2.0,4059014,Maharashtra,Pune,Unknown,18.5312,73.858,1
8,None,None,16d826d5442xd927a85b|1570687938194,India,1000001,NaN,NaN,NaN,None,Current_location,None,Current_location,22.4447,22.4447,1
9,None,None,16d06824ad6xf9735920|1570709506121,Polygon,5455541,17.49914,78.47452,3.0,5455541,Telangana,Hyderabad,Suchitra Road,17.4991,78.4745,1


In [171]:
buyer = dfs.groupby(['lat' , 'long' ,'geography_l1_name_en' , 'geography_l3_name_en', 'locations'])['params_resultset_id'].nunique().sort_values(ascending=False).to_frame().reset_index()

In [286]:
buyer[buyer['lat'] == 28.67]

,lat,long,geography_l1_name_en,geography_l3_name_en,locations,params_resultset_id


In [173]:
buyer.head()

,lat,long,geography_l1_name_en,geography_l3_name_en,locations,params_resultset_id
0,22.4447,22.4447,Current_location,Current_location,India,53281
1,10.8505,76.2711,Kerala,Unknown,Polygon,7928
2,11.1271,78.6569,Tamil Nadu,Unknown,Polygon,6229
3,12.9663,77.5532,Karnataka,Unknown,Polygon,5068
4,19.0591,72.8605,Maharashtra,Unknown,Polygon,4988


In [174]:
buyers_polygon = buyer[buyer['locations'] == 'Polygon'][:1000]

In [189]:
buyers_polygon['params_resultset_id'].iloc[4]

6

In [209]:
df.head()

,map_lat,map_lon,qty_listings,geography_level,geography_l1_name_en,geography_l2_name_en,geography_l3_name_en,total_listings,porcentaje
283117,19.1349,72.8254,4777,3,Maharashtra,Mumbai,Andheri West,3419387,0.00139703
12484,28.6191,77.054,4119,3,Delhi,Delhi,Uttam Nagar,3419387,0.0012046
78094,19.1155,72.8727,4087,3,Maharashtra,Mumbai,Andheri East,3419387,0.00119524
298849,0,0,3819,Not in location tree,Not in location tree,Not in location tree,Not in location tree,3419387,0.00111687
76295,28.6191,77.054,2750,Not in location tree,Not in location tree,Not in location tree,Not in location tree,3419387,0.000804238


In [ ]:
df[df['']]

In [217]:
df1 = df.copy()
df1 = df1[(df1['map_lat'] >= 28.63) & (df1['map_lat'] <= 28.67)] # Arriba a la derecha
df1 = df1[(df1['map_lon'] <= 77.24) & (df1['map_lon'] >= 77.15)] # Arriba a la izquierda
df1 = df1[:1000]

In [236]:
df2.dtypes

lat                     object
long                    object
geography_l1_name_en    object
geography_l3_name_en    object
locations               object
params_resultset_id      int64
dtype: object

In [235]:
df2 = buyer[buyer['locations'] == 'Polygon'].copy()
df2 = df2[(df2['lat'] >= 28.63) & (df2['lat'] <= 28.67)] # Arriba a la derecha
df2 = df2[(df2['long'] <= 77.24) & (df2['long'] >= 77.15)] # Arriba a la izquierda
df2 = df2[:1000]

In [ ]:
buyer[buyer[]]

In [258]:
df2.head()

,lat,long,geography_l1_name_en,geography_l3_name_en,locations,params_resultset_id
11,28.67,77.23,Delhi,Unknown,Polygon,3231
246,28.6524,77.1901,Delhi,Karol Bagh,Polygon,147
1160,28.6483,77.1736,Delhi,Patel Nagar,Polygon,23
1597,28.648,77.2344,Delhi,Jama Masjid,Polygon,16
1601,28.655,77.2337,Delhi,Chandni Chowk,Polygon,16


In [251]:
#

In [248]:
df3 = buyer[buyer['locations'] == 'Current_location'].copy()
df3 = df3[(df3['lat'].astype(float) >= 28.63) & (df3['lat'].astype(float) <= 28.67)] # Arriba a la derecha
df3 = df3[(df3['long'].astype(float) <= 77.24) & (df3['long'].astype(float) >= 77.15)] # Arriba a la izquierda
df3 = df3[:10]

In [ ]:
buyer[buyer[''] == 3231]

In [ ]:
arriba izq 28.917799, 76.736972
arriba derecha 28.883690, 77.508825
abajo derecha 28.355024, 77.509834
abajo izquierda 28.409705, 76.798068

In [276]:
# Make an empty map
m = folium.Map(location=[28.63, 77.19], zoom_start=12)


for lat, lon,traffic_q ,vol, l1, l3 in zip(df1['map_lat'], df1['map_lon'],df1['geography_level'], df1['qty_listings'], df1['geography_l1_name_en'], df1['geography_l3_name_en']):
    folium.CircleMarker(
        [lat, lon],
        radius=vol/10,
        popup = ('L1: ' + str(l1).capitalize() + '<br>'
                 'L3: ' + str(l3) + '<br>'
                 'Listings: ' + str(vol)
                ),
        color='crimson',
        key_on = traffic_q,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[traffic_q],
        fill=True,
        fill_opacity=0.7
        ).add_to(m)
    

for i in range(0,len(df2)):
    folium.CircleMarker(
            [float(df2['lat'].iloc[i]), float(df2['long'].iloc[i])],
            radius=float(df2['params_resultset_id'].iloc[i] / 50),
            popup = ('L1: ' + str(df2['geography_l1_name_en'].iloc[i]).capitalize() + '<br>'
                     'L3: ' + str(df2['geography_l3_name_en'].iloc[i]) + '<br>'
                     'Searches: ' + str(df2['params_resultset_id'].iloc[i])
                    ),
            threshold_scale=[0,1,2,3],
            fill_color='blue',
            fill_opacity=0.7
            ).add_to(m)
    

for i in range(0,len(df3)):
    folium.CircleMarker(
            [float(df3['lat'].iloc[i]), float(df3['long'].iloc[i])],
            radius=float(df3['params_resultset_id'].iloc[i]*2),
            popup = ('L1: ' + str(df3['geography_l1_name_en'].iloc[i]).capitalize() + '<br>'
                     'L3: ' + str(df3['geography_l3_name_en'].iloc[i]) + '<br>'
                     'Searches: ' + str(df3['params_resultset_id'].iloc[i])
                    ),
            threshold_scale=[0,1,2,3],
            fill_color='blue',
            fill_opacity=0.7
            ).add_to(m)

    
m